### The code was imported from the AI in Medicine Specialization Coursera course Assignment 1 Week 1

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2021-11-07 12:11:23.231383: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
train_df = pd.read_csv(
    filepath_or_buffer="../labels/train_u-zeroes.csv",
    dtype={  # Setting labels to type np.int32 was necessary for conversion to tf.Tensor object
        "Path": str,
        "Atelectasis": np.int32,
        "Cardiomegaly": np.int32,
        "Consolidation": np.int32,
        "Edema": np.int32,
        "Pleural Effusion": np.int32,
        "Pleural Other": np.int32,
        "Pneumonia": np.int32,
        "Pneumothorax": np.int32,
        "Enlarged Cardiomediastinum": np.int32,
        "Lung Opacity": np.int32,
        "Lung Lesion": np.int32,
        "Fracture": np.int32,
        "Support Devices": np.int32,
        "No Finding": np.int32
    }
)

In [3]:
list_columns = list(train_df.columns)
y_cols = list_columns[1::]  # First column is 'Path' column

In [4]:
train_datagen = ImageDataGenerator()

In [5]:
train_datagenerator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../dataset/",
    x_col='Path',
    y_col=y_cols,
    weight_col=None,
    target_size=(512, 512),
    color_mode='grayscale',
    class_mode='raw',
    batch_size=16,
    shuffle=True,
    validate_filenames=True
)

Found 223414 validated image filenames.


In [6]:
train_datagenerator.labels

array([[0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

In [7]:
def compute_class_freqs(labels):
    """
    Compute positive and negative frequences for each class.

    Args:
        labels (np.array): matrix of labels, size (num_examples, num_classes)
    Returns:
        positive_frequencies (np.array): array of positive frequences for each
                                         class, size (num_classes)
        negative_frequencies (np.array): array of negative frequences for each
                                         class, size (num_classes)
    """
    # total number of patients (rows).
    N = labels.shape[0]
    
    positive_frequencies = np.sum(labels, axis=0) / N
    negative_frequencies = np.sum(labels==0, axis=0) / N
    
    return positive_frequencies, negative_frequencies

In [8]:
freq_pos, freq_neg = compute_class_freqs(train_datagenerator.labels)

In [9]:
freq_pos

array([0.14939082, 0.12085187, 0.06616864, 0.23385285, 0.3857726 ,
       0.01576893, 0.02703054, 0.08704916, 0.0483318 , 0.47258005,
       0.04111649, 0.040463  , 0.51921992, 0.10017725])

In [10]:
freq_neg

array([0.85060918, 0.87914813, 0.93383136, 0.76614715, 0.6142274 ,
       0.98423107, 0.97296946, 0.91295084, 0.9516682 , 0.52741995,
       0.95888351, 0.959537  , 0.48078008, 0.89982275])

In [11]:
pos_weights = freq_neg
neg_weights = freq_pos

In [12]:
pos_weights

array([0.85060918, 0.87914813, 0.93383136, 0.76614715, 0.6142274 ,
       0.98423107, 0.97296946, 0.91295084, 0.9516682 , 0.52741995,
       0.95888351, 0.959537  , 0.48078008, 0.89982275])

In [13]:
neg_weights

array([0.14939082, 0.12085187, 0.06616864, 0.23385285, 0.3857726 ,
       0.01576893, 0.02703054, 0.08704916, 0.0483318 , 0.47258005,
       0.04111649, 0.040463  , 0.51921992, 0.10017725])

In [14]:
np.save("chexpert_positive_weights.npy", pos_weights)
np.save("chexpert_negative_weights.npy", neg_weights)